In [11]:
# two local csv files loaded from desktop and pandas-dataframed by using one click button
# merge two csv files
# write the merged file to csv format
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_fcf1c90868844bc1ab7d4fffe6063140(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_1fd445c3b67620d21c5874b9c29e9fbf561cebb6','domain': {'id': '20b49a5877434f9486aa2a1d2fcdd21c'},
            'password': 'i&g.LVqvB1Hv_1~X'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_fcf1c90868844bc1ab7d4fffe6063140('newciidemotest', 'cii_input1.csv'))
df_data_2 = pd.read_csv(get_object_storage_file_with_credentials_fcf1c90868844bc1ab7d4fffe6063140('newciidemotest', 'cii_input2.csv'))

# if the matching key, in this case, the INSURANCE_POLICY_ID was read in as a xbf format, you may need to reformat it as follows.
df_data_1.rename(columns={'﻿INSURANCE_POLICY_ID':'INSURANCE_POLICY_ID'}, inplace=True)
df_data_2.rename(columns={'﻿INSURANCE_POLICY_ID':'INSURANCE_POLICY_ID'}, inplace=True)

#df_data_2.head()
m = pd.merge(df_data_1, df_data_2, how = 'inner', on = 'INSURANCE_POLICY_ID')
#m.dtypes

m.to_csv('merge_ab_chyl1.csv', index = False)

In [12]:
#create a put function 

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print resp2

In [13]:
# insert one of the data credentials and modify it with merged file credential
# put the file into the attached Bluemix Obj Storage (Swift obj storage)

credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_fcf1c908_6884_4bc1_ab7d_4fffe6063140',
  'project_id':'f1a0c90407004288832edb2eebb37f36',
  'region':'dallas',
  'user_id':'7b5db9aabbb34e558bf778a623ce0106',
  'domain_id':'20b49a5877434f9486aa2a1d2fcdd21c',
  'domain_name':'1316891',
  'username':'member_1fd445c3b67620d21c5874b9c29e9fbf561cebb6',
  'password':"""i&g.LVqvB1Hv_1~X""",
  'container':'newciidemotest',
  'tenantId':'undefined',
  'filename':'merge_ab_chyl1.csv'
}

put_file(credentials_1, 'merge_ab_chyl1.csv')

# if put function works, the output of <Response [201]> will show up

<Response [201]>
